# Using joblib to cache slow computations (for science)

If you'll indulge some CS jargon, I want to talk a little bit about **pure functions** and **memoization**.

**Pure functions** are functions where, if I provide the same inputs, I always get the same outputs (and there are no side-effects). Mathematical functions are a good example. `sin(0)` is always `0`, and nothing squirrely is happening behind the scenes (like writing the output to a file, or emailing your professor to shame you for having to calculate `sin(0)` on a computer).

Pure functional code has many nice properties. For parallel / distributed workloads, you only have to replicate the function inputs on your worker nodes (rather than the entire program state, or even the entire computer state).

Another nice property is that pure functions are trivially **memoized**, or cached. This means you can replace a function call with a lookup from some stored outputs without changing the meaning of the program, so long as the inputs to that function are present among the stored outputs.

Let's look at `sin(x)`, for example.

In [3]:
from math import sin
print(sin.__doc__)

sin(x)

Return the sine of x (measured in radians).


The `sin(x)` function is pretty fast, since a lot of effort has gone into optimizing it:

In [4]:
%timeit sin(0.1)

103 ns ± 4.92 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Suppose it's the 16th century and `sin(0.1)` takes unreasonably long to compute on the fly. We'll emulate this by defining `sine(x)` (since we hate abbreviations) and inserting a delay.

In [5]:
import time
def sine(x):
    time.sleep(0.2)
    return sin(x)

In [6]:
%timeit sine(0.1)

205 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


How slow.

Well, if we know we're using `sin(0.1)` a lot, we can just store its value in a variable. Variable access is so fast it's practically instant, compared to calling that function.

In [7]:
sine_one_tenth = sine(0.1)

But then you have to remember whether to call the function or use the variable. What if we just hid that detail away?

In [8]:
def new_sine(x):
    if x == 0.1:
        return sine_one_tenth
    else:
        return sine(x)

In [9]:
%timeit new_sine(0.1)

185 ns ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Note that our new function still calls `sine(x)` for arguments that we don't have cached (that is, anything but `0.1`). They will be much slower to compute, as expected:

In [10]:
%timeit new_sine(0.2)

203 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Suppose we wanted to support more than just `sin(0.1)` on the "fast path".

We can set up a dictionary so that, when there's a value we haven't seen before, we call `sin(x)` and store the output in the dictionary.

In [11]:
_previously_seen_arguments = {}
def new_sine(x):
    if x not in _previously_seen_arguments:
        _previously_seen_arguments[x] = sine(x)
    return _previously_seen_arguments[x]

In [12]:
%timeit new_sine(0.1)

2.44 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit new_sine(0.2)

The slowest run took 5.57 times longer than the fastest. This could mean that an intermediate result is being cached.
2.39 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


For the sake of demonstration, I slowed the `sin(x)` function down orders of magnitude. Otherwise, our piddly Python caching wrapper would be way slower than just calling `sin(x)` every time.

There's a lesson in there: **You will only want to cache outputs when a function takes so long to run that it's worth the overhead.**

You'll develop an intuition for things that are likely to be slow (e.g. repeatedly hitting the disk or network for data) vs. fast (e.g. arithmetic and optimized mathematical functions). There's also `%timeit`, as well as a whole set of tools to find the bottlenecks in your code (the art of which is called "profiling").

# What does any of this have to do with `joblib`?

The great thing about `joblib` is that it has a general-case solution to the "building a `_previously_seen_arguments` dictionary" problem.

(The Python standard library has one too, but it's poorly suited for scientific applications.)

In [63]:
from joblib import Memory
cachedir = 'cache'
memory = Memory(cachedir=cachedir)

The `memory` object we just created is our reference to the `joblib` machinery we initialized to save ourselves from rewriting variations on that last function over and over again.

Let's see what `new_sine()` looks like, `joblib` style:

In [64]:
@memory.cache
def new_sine2(x):
    return sine(x)

In [65]:
%timeit new_sine2(0.1)

799 µs ± 44.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [66]:
%timeit new_sine2(0.2)

929 µs ± 42.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Note that the overhead is much _greater_ compared to the simplistic Python implementation I wrote, but _still_ orders of magnitude faster than `sine(x)`.

### Aside: decorators

If you've never seen it before, that `@memory.cache` line may seem to be magic. For most cases, you can think of it that way, but it may occasionally help you to know what it actually _means_.

That notation is called a "function decorator" or just a "decorator". You could say that you are "decorating" `new_sine()`, or that `memory.cache` is a "decorator" that is being applied to `new_sine`.

We could equivalently write:

In [73]:
def new_sine2(x):
    return sine(x)

new_sine2 = memory.cache(new_sine2)

In [74]:
%timeit new_sine2(0.1)

________________________________________________________________________________
[Memory] Calling __main__--Users-jdl-Dropbox-software-code-coffee-downloads-2017-18-__ipython-input__.new_sine2...
new_sine2(0.1)
________________________________________________________new_sine2 - 0.2s, 0.0min
1.01 ms ± 333 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
%timeit new_sine2(0.2)

________________________________________________________________________________
[Memory] Calling __main__--Users-jdl-Dropbox-software-code-coffee-downloads-2017-18-__ipython-input__.new_sine2...
new_sine2(0.2)
________________________________________________________new_sine2 - 0.2s, 0.0min
937 µs ± 400 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


So, `memory.cache` is a function, that's being passed a function... and returning a function! At the end of this cell, the `new_sine` variable holds a function that, when called:

* Checks if the arguments supplied match any its seen before
* If yes, loads the resulting value and returns it
* If no, calls the original `new_sine` function, adds the arguments and return value to the list of inputs and outputs its seen, and returns the value

The `joblib` machinery takes care of a lot of fiddly details for us. Let's peek into `cachedir`, which we passed in when we initialized the `memory` instance:

In [76]:
ls $cachedir/

joblib/


Hm, just a `joblib` folder... wonder what's in there.

(The `$cachedir` notation is IPython-specific: it means "substitute this Python variable from the current session into my shell command.")

In [77]:
ls $cachedir/joblib/

__main__--Users-jdl-Dropbox-software-code-coffee-downloads-2017-18-__ipython-input__/
cacheable_functions/
pipeline/


That's the full path to the code (or would be, if it were in a separate file rather than a notebook cell). So, functions with the same name in different files won't interfere with each other in the cache.

In [78]:
ls $cachedir/joblib/__main__--*/

new_sine/  new_sine2/


In [79]:
ls $cachedir/joblib/__main__--*/new_sine2/

7d6f6f8bfe705c3506326eae7f01d8a7/ func_code.py
f48084a3ed562f85c12b64c439485607/


In [80]:
ls $cachedir/joblib/__main__--*/new_sine2/f48084a3ed562f85c12b64c439485607

metadata.json  output.pkl


In [81]:
cat $cachedir/joblib/__main__--*/new_sine2/f48084a3ed562f85c12b64c439485607/metadata.json

{"duration": 0.21219372749328613, "input_args": {"x": "0.1"}}

In [82]:
cat $cachedir/joblib/__main__--*/new_sine2/func_code.py

# first line: 1
def new_sine2(x):
    return sine(x)


Hey, there's the exact code we decorated! And there's a couple of folders with weird hex string names. That's the [hash](https://en.wikipedia.org/wiki/Hash_function) of the function arguments, which joblib uses to find where it saved the cached output. (This is just like using the arguments as a dictionary key, but more general.)

The function code is saved in order to tell when the function body (`def new_sine2(x): [...]`) changes. This ensures you don't see a mix of new and stale results when you update the definition of your function.

The `joblib` wrapper handles a bunch of stuff I didn't include in my demo version of a function cache:

  * Wrapping functions with multiple arguments
  * Persisting the cache to disk so it remains after you close and restart Python
  * Clearing the cache if the function body changes
  * Converting NumPy arrays to a hashable representation

Basically, it's the general case solution to caching outputs of slow functions. As promised.

# Important caveats

The ability of `joblib` to tell when two functions are the same (and whether it should reuse cached outputs) is compromised by defining functions in notebooks. You can still _run_ the functions in a notebook, but for best cache usage I always place functions that I am "done" writing (at least temporarily) in an external Python module like `pipeline.py`.

Let's see what the cache looks like for that case, using the [`%%writefile` magic](http://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) to make a `pipeline.py`:

In [59]:
%%writefile pipeline.py
import joblib
import time
from math import sin

memory = joblib.Memory(cachedir='./cache')

@memory.cache
def slow_sine(x):
    time.sleep(0.2)
    return sin(x)

@memory.cache
def foo(x):
    return x + 1

Overwriting pipeline.py


In [60]:
from pipeline import slow_sine

In [61]:
slow_sine(0.1)

0.09983341664682815

In [62]:
ls $cachedir/joblib/

__main__--Users-jdl-Dropbox-software-code-coffee-downloads-2017-18-__ipython-input__/
cacheable_functions/
pipeline/


The `pipeline/` folder has cache outputs for all the functions defined in the `pipeline` module. (Much nicer than the complicated `__ipython-input__` naming for functions defined in a notebook.)

In [83]:
ls $cachedir/joblib/pipeline/slow_sine/

f48084a3ed562f85c12b64c439485607/ func_code.py


Another important caveat is that the cache size can grow unbounded. The `bytes_limit=` argument to `Memory()` can be used to limit the total size of the cache; when the total size of the fies in cache approaches the limit, the least-recently used outputs are deleted to keep the cache directory under a certain size.

# When would I need this in astronomy?

I came across this functionality when I was doing some exploratory analysis with machine learning. It would take minutes to recompute when I tweaked some input parameter, and I was pretty frequently adjusting a parameter only to ultimately put it back... and have to recompute a slow function that I had definitely already computed.

I ended up splitting my code into a function that did the decomposition I wanted, and a few lines to plot the result. The function, I decorated with `memory.cache`. Then, I only redid the computation when I actually needed to.